In [2]:
from pseudo_povm_circuits import *

In [3]:
def generate_random_pseudo_povm():
  '''Generates random pseudo POVM

  Returns instance of PseudoPovm class
  '''
  
  povm_ten = tf.random.normal([4, 3], dtype = tf.float64)
  return PseudoPovm(povm_ten)

In [4]:
def total_negativity_optimization(circuit):
  '''Finds pseudo POVM where total negativity of given circuit is minimal

  Arguments
  ---------
  circuit : QubitCircuit 

  Returns instance of class PseudoPovm
  '''


  #=========================#
  lr = 0.02 # learning rate
  #=========================#
  opt = tf.keras.optimizers.Adam(lr)

  def neg(var):
    povm = PseudoPovm(var)
    return circuit.total_negativity(povm) 

  povm = generate_random_pseudo_povm()


  var = povm.lam
  var = tf.Variable(var)


  #print(neg(var))

  for i in range(1000):
    with tf.GradientTape() as tape:

      loss = neg(var)
    grad = tape.gradient(loss, var) 
    opt.apply_gradients(zip([grad], [var]))  # minimization step
  
  lam = tf.complex(var,tf.zeros([4, 3], dtype = tf.float64))
  return PseudoPovm(lam)

In [5]:
# Initialize some single qubit gates
X = sigma_x
Y = sigma_y
Z = sigma_z
T = tf.constant([[1 ,0],[0, cmath.exp(1j * math.pi / 4)]], dtype=tf.complex128)
H = tf.constant([[1,1],[1,-1]], dtype=tf.complex128) / math.sqrt(2)
S = tf.constant([[1,0],[0,1j]],dtype=tf.complex128)

In [6]:
# Create pseudo POVM
povm = generate_random_pseudo_povm()

In [7]:
povm.get_pseudo_povm_in_comp_basis()

<tf.Tensor: shape=(4, 2, 2), dtype=complex128, numpy=
array([[[-2.80660541+0.j        , -0.83844745-0.84832814j],
        [-0.83844745+0.84832814j,  0.16217345+0.j        ]],

       [[-1.12653776+0.j        ,  1.68264056+0.52501243j],
        [ 1.68264056-0.52501243j,  2.00688917+0.j        ]],

       [[-0.85176496+0.j        , -0.34920447+0.06039737j],
        [-0.34920447-0.06039737j, -0.82237792+0.j        ]],

       [[ 5.78490814+0.j        , -0.49498864+0.26291833j],
        [-0.49498864-0.26291833j, -0.3466847 +0.j        ]]])>

In [8]:
# Create quantum circuit
circuit = QubitCircuit([X,Y,H])

In [9]:
# Run circuit 
circuit.run_circuit(povm)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 0.5])>

In [10]:
# Check total negativity before optimization
circuit.total_negativity(povm)

<tf.Tensor: shape=(), dtype=float64, numpy=19.34422559578541>

In [11]:
# Find pseudo POVM where negativity is minimal
optimal_basis = total_negativity_optimization(circuit)

In [12]:
# Let's check total negativity in nev pseudo POVM
circuit.total_negativity(optimal_basis)

<tf.Tensor: shape=(), dtype=float64, numpy=2.1163490295629472>